In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import warnings
warnings.filterwarnings('ignore')

# ------------------- caminhos dos arquivos
TRAIN_PATH = '/kaggle/input/campeonato-inteli-modulo3-2025/train.csv'
TEST_PATH = '/kaggle/input/campeonato-inteli-modulo3-2025/test.csv'
SAMPLE_PATH = '/kaggle/input/campeonato-inteli-modulo3-2025/sample_submission.csv'

print("=" * 70)
print("CARREGANDO DADOS")
print("=" * 70)

# -------------------- Distribuição da variável alvo
plt.figure(figsize=(5,4))
sns.countplot(x=df['labels'])
plt.title("Distribuição da variável alvo (labels)")
plt.show()
print("Taxa de sucesso:", df['labels'].mean())


## importancia das features
# Pega os coeficientes da primeira camada do MLP
first_layer_weights = model_kaggle.coefs_[0]

# Calcula a importância como a soma absoluta dos pesos conectados a cada feature
feature_importance = np.sum(np.abs(first_layer_weights), axis=1)

# Normaliza para somar 1
feature_importance = feature_importance / np.sum(feature_importance)

# Coloca em um DataFrame para organizar
feat_importances = pd.DataFrame({
    'Feature': X_train[features].columns,
    'Importance': feature_importance
}).sort_values(by='Importance', ascending=False)

# Plot
plt.figure(figsize=(8,5))
sns.barplot(x='Importance', y='Feature', data=feat_importances, palette="viridis")
plt.title("Importância das Features no MLP (baseada nos pesos da 1ª camada)")
plt.show()

print("\nTop features mais relevantes para o MLP:")
print(feat_importances.head(10))

# jogando os dados pro dataframe e já tirando duplicados
df = pd.read_csv(TRAIN_PATH)
df = df.drop_duplicates()

print(f"✓ Dados carregados: {len(df)} linhas")

# bora ver os nulos
print("\nValores nulos por coluna:")
print(df.isna().sum())

# arrumando os nulos
print("\n" + "=" * 70)
print("LIMPANDO VALORES NULOS")
print("=" * 70)

df['age_first_funding_year'].fillna(0, inplace=True)
df['age_first_milestone_year'].fillna(0, inplace=True)
df['age_last_funding_year'].fillna(df['age_first_funding_year'], inplace=True)
df['age_last_milestone_year'].fillna(df['age_first_milestone_year'], inplace=True)

print("✓ Valores nulos tratados")
print("\nNulos restantes:")
print(df.isna().sum().sum())

# pegar só as numéricas que fazem sentido normalizar
numerical_cols = df.select_dtypes(include='number').columns
filtered_cols = [col for col in numerical_cols if not (col.startswith('is_') or col.startswith('has_'))]
filtered_cols.remove('labels')
filtered_cols.remove('id')

print(f"\nColunas numéricas para normalizar: {len(filtered_cols)}")

# criar uma cópia só pro treino
df_train = df.copy()

# normalizando os dados
print("\n" + "=" * 70)
print("NORMALIZANDO DADOS")
print("=" * 70)

standard = StandardScaler()
df_train[filtered_cols] = standard.fit_transform(df_train[filtered_cols])

print("✓ Dados normalizados com StandardScaler")

# transformando categoria em colunas (One-Hot)
print("\n" + "=" * 70)
print("APLICANDO ONE-HOT ENCODING")
print("=" * 70)

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_train = encoder.fit_transform(df_train[['category_code']])
encoded_train_df = pd.DataFrame(
    encoded_train,
    columns=encoder.get_feature_names_out(['category_code']),
    index=df_train.index
)

df_train = pd.concat([df_train.drop(columns=['category_code']), encoded_train_df], axis=1)

print(f"✓ One-Hot Encoding aplicado: {encoded_train.shape[1]} novas colunas")

# tratando os outliers do funding usando IQR
Q1 = df_train['funding_total_usd'].quantile(0.25)
Q3 = df_train['funding_total_usd'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_train_capped = df_train.copy()
df_train_capped['funding_total_usd'] = np.where(
    df_train_capped['funding_total_usd'] < lower_bound, lower_bound,
    np.where(df_train_capped['funding_total_usd'] > upper_bound, upper_bound,
             df_train_capped['funding_total_usd'])
)

print(f"✓ Outliers tratados (IQR method)")

# separando features e target
X_train = df_train.drop('labels', axis=1)
y_train = df_train['labels']

print(f"\n✓ X_train: {X_train.shape}")
print(f"✓ y_train: {y_train.shape}")
print(f"✓ Taxa de positivos: {y_train.mean():.2%}")

# features que escolhi pra treinar
features = [
    'age_first_funding_year',
    'age_last_milestone_year',
    'age_last_funding_year',
    'milestones',
    'funding_rounds',
    'funding_total_usd',
    'relationships',
    'is_ecommerce'
]

print("\n" + "=" * 70)
print("FEATURES SELECIONADAS")
print("=" * 70)
for i, feat in enumerate(features, 1):
    print(f"  {i}. {feat}")

# bora treinar a MLP
print("\n" + "=" * 70)
print("TREINANDO MODELO MLP")
print("=" * 70)

model_kaggle = MLPClassifier(
    hidden_layer_sizes=(100, 50),
    activation='tanh',
    solver='sgd',
    learning_rate_init=0.01,
    alpha=0.001,
    max_iter=500,
    random_state=18
)

print("Configuração do modelo:")
print(f"  Hidden layers: (100, 50)")
print(f"  Activation: tanh")
print(f"  Solver: sgd")
print(f"  Learning rate: 0.01")
print(f"  Alpha (L2): 0.001")
print(f"  Max iterations: 500")
print(f"  Random state: 18")

model_kaggle.fit(X_train[features], y_train)

print("\n✓ Modelo treinado!")

# testando no próprio treino só pra ver
y_pred = model_kaggle.predict(X_train[features])
train_acc = accuracy_score(y_train, y_pred)
cm = confusion_matrix(y_train, y_pred)

print("\n" + "=" * 70)
print("RESULTADOS NO TREINO")
print("=" * 70)
print(f"Acurácia: {train_acc:.4f} ({train_acc*100:.2f}%)")
print("\nMatriz de Confusão:")
print(cm)
print(f"\nTrue Negatives:  {cm[0,0]}")
print(f"False Positives: {cm[0,1]}")
print(f"False Negatives: {cm[1,0]}")
print(f"True Positives:  {cm[1,1]}")

# agora preparar o dataset de teste
print("\n" + "=" * 70)
print("PREPARANDO DADOS DE TESTE")
print("=" * 70)

df_final = pd.read_csv(TEST_PATH)
print(f"✓ Teste carregado: {len(df_final)} linhas")

# normalizar igual no treino
df_final[filtered_cols] = standard.transform(df_final[filtered_cols])
print("✓ Teste normalizado")

# aplicar o One-Hot no teste também
encoded_final = encoder.transform(df_final[['category_code']])
encoded_final_df = pd.DataFrame(
    encoded_final,
    columns=encoder.get_feature_names_out(['category_code']),
    index=df_final.index
)

df_final = pd.concat([df_final, encoded_final_df], axis=1)
df_final.drop('category_code', axis=1, inplace=True)
print("✓ One-Hot Encoding aplicado no teste")

# arrumando nulos no teste também
df_final['age_first_funding_year'].fillna(0, inplace=True)
df_final['age_first_milestone_year'].fillna(0, inplace=True)
df_final['age_last_funding_year'].fillna(df_final['age_first_funding_year'], inplace=True)
df_final['age_last_milestone_year'].fillna(df_final['age_first_milestone_year'], inplace=True)
print("✓ Valores nulos tratados no teste")

# gerar as previsões finais
print("\n" + "=" * 70)
print("GERANDO PREDIÇÕES FINAIS")
print("=" * 70)

y_test_pred = model_kaggle.predict(df_final[features])

# salvar o arquivo de submissão
submission = pd.DataFrame({
    'id': df_final['id'],
    'labels': y_test_pred
})

submission.to_csv('submissionGAP1.csv', index=False)

print(f"✓ Predições geradas: {len(y_test_pred)}")
print(f"  Positivos: {y_test_pred.sum()} ({y_test_pred.mean():.1%})")
print(f"  Negativos: {len(y_test_pred) - y_test_pred.sum()} ({1 - y_test_pred.mean():.1%})")
print(f"\n✅ submission_mlp.csv salvo com sucesso!")
print(f"\n🎯 Acurácia esperada no leaderboard: ~{train_acc:.1%}")
print("=" * 70)


# ==============================================
# FORMULAÇÃO DE HIPÓTESES E ANÁLISE EXPLORATÓRIA
# ==============================================
print("\n" + "=" * 70)
print("EXPLORAÇÃO DE DADOS E HIPÓTESES")
print("=" * 70)

# Distribuição das labels
plt.figure(figsize=(5,4))
sns.countplot(x=df['labels'])
plt.title("Distribuição das Labels (0 = não sucesso, 1 = sucesso)")
plt.show()

print(f"Taxa de sucesso no dataset: {df['labels'].mean():.2%}")

# ---------------------------------------------------
# Hipótese 1 – Maior funding aumenta chance de sucesso
# ---------------------------------------------------
print("\nHipótese 1: Startups com maior funding_total_usd têm maior chance de sucesso")

plt.figure(figsize=(7,5))
sns.boxplot(x='labels', y='funding_total_usd', data=df)
plt.yscale("log")  # escala log pq funding é muito assimétrico
plt.title("Funding Total (USD) vs Sucesso")
plt.show()

print("Médias de funding por label:")
print(df.groupby('labels')['funding_total_usd'].mean())

# ---------------------------------------------------
# Hipótese 2 – Mais rodadas de investimento aumentam sucesso
# ---------------------------------------------------
print("\nHipótese 2: Startups com mais funding_rounds têm maior chance de sucesso")

plt.figure(figsize=(7,5))
sns.boxplot(x='labels', y='funding_rounds', data=df)
plt.title("Funding Rounds vs Sucesso")
plt.show()

print("Média de rounds por label:")
print(df.groupby('labels')['funding_rounds'].mean())

# ---------------------------------------------------
# Hipótese 3 – Categoria influencia o sucesso
# ---------------------------------------------------
print("\nHipótese 3: O setor (category_code) afeta a probabilidade de sucesso")

# calcular taxa de sucesso por categoria
cat_success = df.groupby('category_code')['labels'].mean().sort_values(ascending=False).head(10)

plt.figure(figsize=(10,5))
sns.barplot(x=cat_success.index, y=cat_success.values)
plt.xticks(rotation=45, ha="right")
plt.title("Top 10 Categorias com Maior Taxa de Sucesso")
plt.ylabel("Taxa de Sucesso")
plt.show()

print("Taxa de sucesso média por categoria (top 10):")
print(cat_success)



# Matriz de confusão
cm = confusion_matrix(y_train, y_pred)

plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt='d', cmap="Blues", xticklabels=[0,1], yticklabels=[0,1])
plt.title("Matriz de Confusão - MLP (Treino)")
plt.xlabel("Previsto")
plt.ylabel("Real")
plt.show()

print("Relatório de Classificação:\n")
print(classification_report(y_train, y_pred))